# Some considerations

- Neural Inpainting : on actual license plates, its probable that due to noise/perspective etc. , when we extract single characters, parts of the characters will be missing. We could try to have a neural inpainting type of model, where we hide parts of the characters in the synthetic dataset and train the model to reconstruct these



- In this code, I will consider a variational AE that should reconstruct the actual characters based on corrupted/noisy/partial characters